# MATLAB pre-processing routine

#### _Set the experiment number:_

In [ ]:
expnr = '364';

# Setup

In [ ]:
format compact

In [ ]:
% set paths
DA_EXPDIR = getenv('DA_EXPDIR');
DA_TOPDIR = getenv('DA_TOPDIR');
DA_PREDIR = [DA_TOPDIR '/pre-post/pre/'];
addpath(genpath(DA_PREDIR));   % add all subdirectories to path
exppath = [DA_EXPDIR '/'];

## Set directory to save outputs

In [ ]:
% set directory to save output
savedir = [DA_EXPDIR '/' expnr '/'];
% save new or overwrite
saveasnew = false;
saveold = true;

In [ ]:
starttime = datetime;

if saveold
    olddir = [savedir 'old/' datestr(starttime,'yyyy-mm-dd') '-' datestr(starttime,'HH-MM')];
    mkdir(olddir)
    copyfile(blocksfile, [olddir '/blocks.inp.' expnr])
    copyfile([savedir '/xgrid.inp.' expnr], [olddir '/xgrid.inp.' expnr])
    copyfile([savedir '/zgrid.inp.' expnr], [olddir '/zgrid.inp.' expnr])
end

if saveasnew
    outputdir = [savedir 'output/' datestr(starttime,'yyyy-mm-dd') '-' datestr(starttime,'HH-MM')];
    tempdir = [savedir 'temp/' datestr(starttime,'yyyy-mm-dd') '-' datestr(starttime,'HH-MM')];
    mkdir(outputdir)
    mkdir(tempdir)
else
    outputdir = savedir;
    tempdir = [savedir 'temp/'];
    mkdir(tempdir)
end

clear starttime

## Setup parameters

In [ ]:
source = 1;
stretch = 'no';
lwritefiles = 1;
ltestplot = 1;
lhqplot = 0;
lradiation = 0;

In [ ]:
%maximum size of floors and bounding walls (cells in each dimension)
if lradiation
    maxsize = 10; %ADD TO NAMOPTIONS EVENTUALY
else
    maxsize = inf; 
end

In [ ]:
solaz = 30; % azimuth angle
Z = 45; % zenith angle
delta = 0.01; % small adjustment for position of rad corners
centerweight = 0.5;
cornerweight = (1-centerweight)/4;
I = 800; % Direct solar irradiation [W/m2]
Dsk = 300; % Diffuse incoming radiation [W/m2]
pad=0;

---
## Read in base configuration

In [ ]:
% read namoptions
r = da_pp(expnr,exppath);
% read blocks
% blocksfile = [DA_EXPDIR '/' expnr '/old-blocks.' expnr];
blocksfile = [DA_EXPDIR '/' expnr '/blocks.inp.' expnr];
blocks = dlmread(blocksfile,'',2,0);
% copyfile(blocksfile, [DA_EXPDIR '/' expnr '/old-blocks.' expnr])

In [ ]:
% recreate matlab variables needed for Ivo's preproc
dx = r.xsize/r.imax;
dy = r.ysize/r.jtot;
dz = r.zsize/r.kmax;
dh = r.zsize;
dh = 180;
ni = r.imax; nj = r.jtot; nk = r.kmax;

---
## Generate new inps files

In [ ]:
parentdir = DA_PREDIR;
cd(parentdir)

### Walltypes

In [ ]:
% copy default walltypes
copyfile('default/walltypes.inp.xxx',[outputdir '/walltypes.inp.' num2str(expnr)])

### Grids

In [ ]:
%% make grids
makexygrid
makezgrid

### Masking matrix with block heights

In [ ]:
%% make matrix with block heights
topomask = zeros(nj,ni);
topo = zeros(nj,ni);
for n = 1:size(blocks,1)
    topo(blocks(n,1):blocks(n,2),blocks(n,3):blocks(n,4)) = zh(blocks(n,6)+1);
    topomask(blocks(n,1):blocks(n,2),blocks(n,3):blocks(n,4)) = 1;
end

In [ ]:
if ~lradiation
    
    makefacets
    block2fac
    nwallfcts = 0;
    createfloors
    
else
    
    makefacets
    block2fac
    addboundingwalls
    createfloors
    addboundingwalls
    vsolcalc
    vfc
    rayit
    
end

## ! Now update nblocks and nfcts in namoptions !

### Initial temperatures at the wall

In [ ]:
% Manually make Tfacinit
Tnew = ones(nfcts,1)*288;
fname = [outputdir '/Tfacinit.inp.' expnr];
fileID = fopen(fname,'W');
fprintf(fileID, '# Initial facet tempereatures in radiative equilibrium\n');
fclose(fileID);
dlmwrite(fname,Tnew(:,1),'-append','delimiter',' ','precision','%4f')

In [ ]:
% %% Manually make facetarea
% 
% fname = [outputdir '/facetarea.inp.' num2str(expnr)];
% fileID = fopen(fname,'W');
% fprintf(fileID, '# area of facets\n');
% fclose(fileID);
% dlmwrite(fname,A,'-append','delimiter',' ','precision','%4f')